#TEST 1

Analyse de Crédit et Modélisation Prédictive

En France comme à l'étranger, l'utilisation des cartes de crédit pour des achats de consommation ou d'équipement est courante. Les établissements financiers encouragent cette tendance en augmentant le nombre de cartes bancaires avec crédit revolving. Cependant, le taux de défaut sur ces crédits a augmenté avec la crise économique récente.

Notre mission est de développer un modèle prédictif pour identifier les clients susceptibles d'avoir des défauts de paiement sur leur carte de crédit.


Etape 1 : Importation des Bibliothèques Nécessaires


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


Etape 2 : Importation des Bibliothèques Nécessaires, division des données en ensembles d'entraînement et de test, et normalisation des caractéristiques. La normalisation est essentielle pour que les caractéristiques soient sur une même échelle et pour améliorer la performance des algorithmes de machine learning




In [ ]:
# Remplacez 'chemin_vers_votre_fichier' par le chemin réel de votre fichier CSV
data = pd.read_csv('training.csv', sep=';')

# Séparation des caractéristiques et de la cible
X = data.drop(['ID', 'DEF'], axis=1)
y = data['DEF']

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des caractéristiques
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Etape 3 : Optimisation des hyperparamètres avec recherche aléatoire


In [ ]:
# Configuration des paramètres pour la recherche aléatoire
param_distributions = {
    'n_estimators': [100, 200],  # moins de valeurs
    'max_depth': [None, 10, 20],  # moins de valeurs
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [ ]:
# Initialisation du modèle
rf = RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Recherche aléatoire avec moins de validations croisées et moins d'itérations
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=10,  # moins d'itérations
    cv=2,  # moins de validations croisées
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

Etape 4 : Entraînement du modèle et sélection des meilleurs paramètres


In [ ]:
# Entraînement
random_search.fit(X_train_scaled, y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [100, 200]},
                   scoring='accuracy', verbose=2)

In [ ]:
# Affichage des meilleurs paramètres
print(f"Meilleurs paramètres : {random_search.best_params_}")

Meilleurs paramètres : {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10}


Etape 5 : Évaluation des performances du modèle


In [ ]:
# Utilisation du meilleur modèle trouvé par la recherche aléatoire
best_model = random_search.best_estimator_

In [ ]:
# Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test_scaled)

In [ ]:
# Calcul et affichage de l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitude du modèle optimisé : {accuracy}")

Exactitude du modèle optimisé : 0.8220833333333334


Etape 6 : Préparation de l'ensemble d'évaluation et création du fichier de soumission


In [ ]:
donnees_evaluation = pd.read_csv('evaluation.csv', sep=';')


In [ ]:
# Appliquer la même transformation (normalisation) aux caractéristiques du jeu de données d'évaluation
X_evaluation = donnees_evaluation.drop(['ID', 'DEF'], axis=1, errors='ignore')
X_evaluation_scaled = scaler.transform(X_evaluation)


In [ ]:
predictions = best_model.predict(X_evaluation_scaled)

In [ ]:
# Création du fichier de soumission
soumission = pd.DataFrame({'ID': donnees_evaluation['ID'], 'DEF': predictions})

In [ ]:
# Sauvegarde du fichier de soumission au format CSV
soumission.to_csv('soumission.csv', index=False, sep=';')

In [ ]:
from google.colab import files
files.download('soumission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>